# Statoil/C-CORE Iceberg Classifier Challenge 

#### Kaggle kernel 필사하기 - 2
Link: <https://www.kaggle.com/cbryant/keras-cnn-statoil-iceberg-lb-0-1995-now-0-1516>

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import cv2 # Used to manipulated the images 
np.random.seed(0)

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

from keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

In [2]:
train = pd.read_json('train.json')
#test = pd.read_json('test.json')
target = train['is_iceberg']

In [4]:
print(train.shape)

(1604, 5)


## Image data Preprocessing
본 커널에서는 이미지 데이터를 전처리를 진행하고 모델링을 진행했다. 이미지 데이터 전처리 방법은 다음과 같다.
 1. `band_1`, `band_2` 분리하여 독립적인 이미지 데이터를 만들고 `band_1`과 `band_2`의 데이터 값을 합친 이미지인 `band_3`도 생성한다.
 2. 이미지 데이터 값의 범위를 scaling하여 모델이 좀 더 robust 하도록 만든다. 이 커널에서는 Standardizaton(표준화)를 진행했다.

In [5]:
# image data scaling 함수 정의
def get_scaled_img(data):
    imgs = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2
        
        # scaling
        sc1 = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        sc2 = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        sc3 = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())
        
        imgs.append(np.dstack((sc1, sc2, sc3)))
        
    return np.array(imgs)

In [6]:
X_train = get_scaled_img(train)

In [7]:
y_train = np.array(train['is_iceberg'])

## Image data augmentation
CNN 모델링을 할 때, 주어진 이미지로만 학습을 하면 데이터가 부족한 경우도 있고, 이미지의 각도, 색, 위치가 다양하기 때문에 일관된 학습을 하기 힘들다는 단점이 있다. 이 문제를 보완하기 위해 본 커널에서는 **image data augmentation**을 진행했다.

Data augmentation 기법은 매우 다양하다. 대표적으로 이미지를 뒤집는 flip, 회전시키는 rotation, 색을 모두 회색으로 바꾸는 gray scale 등등이 있다. 본 커널에서는 수직, 수평으로 뒤집은 이미지를 추가로 더해주어 데이터를 증강했다.

 + 사용 패키지: `opencv` 
 
 
cf) 본 커널에서는 data augmentation을 위해 opencv 패키지를 사용했지만 tf.keras를 이용해 augmentation을 할 수 있다. 이 부분에 대한 코드도 따로 찾아보고 공부해볼 예정이다.

In [9]:
# image data augmentation - vertical/horizontally flipped images
def get_more_imgs(imgs):
    more_imgs = []
    vert_flip_imgs = []
    horz_flip_imgs = []
    
    for i in range(imgs.shape[0]):
        img1 = imgs[i, :, :, 0]
        img2 = imgs[i, :, :, 1]
        img3 = imgs[i, :, :, 2]
        
        # cv2.flip(a, 1): vertical line 기준으로 뒤집음, cv2.flip(a, 0): horizontal line 기준으로 뒤집음
        img1_v = cv2.flip(img1, 1)
        img1_h = cv2.flip(img1, 0)
        img2_v = cv2.flip(img2, 1)
        img2_h = cv2.flip(img2, 0)
        img3_v = cv2.flip(img3, 1)
        img3_h = cv2.flip(img3, 0)
        
        vert_flip_imgs.append(np.dstack((img1_v, img2_v, img3_v)))
        horz_flip_imgs.append(np.dstack((img1_h, img2_h, img3_h)))
    v = np.array(vert_flip_imgs)
    h = np.array(horz_flip_imgs)
    
    more_imgs = np.concatenate((imgs, v, h))
    
    return more_imgs

In [11]:
X_new_train = get_more_imgs(X_train)
print(X_new_train.shape)

(4812, 75, 75, 3)


In [13]:
y_new_train = np.concatenate((y_train, y_train, y_train))
print(y_new_train.shape)

(4812,)


## CNN Model
모델링 과정에 대한 공부 내용은 앞선 [필사](https://github.com/hyewonleess/Kaggle/blob/master/iceberg_classification/%EC%BB%A4%EB%84%90%ED%95%84%EC%82%AC/iceberg_follow_1.ipynb) 에서 자세히 다뤘으므로 넘어간다.

In [16]:
def getModel():
    #Build keras model
    
    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(lr=0.001, decay=0.0)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

### Callbacks(콜백)
이번 커널에서도 callback이 등장했다. EarlyStopping과 ModelCheckpoint에 대해서는 이전 커널 필사에서 다뤘는데, 이번 커널에서는 `ReduceOnPlateu`가 새롭게 등장했다. 

#### What is ReduceOnPlateu?
`ReduceOnPlateu`는 딥러닝 모델이 학습을 할 때 손실함수의 local minimum에 빠져버려 모델의 개선이 없을 경우, 학습률(learning rate)를 조절해 local minimum에서 빠져나오도록 하는 함수이다. 즉, 모델 성능이 개선이 더 이상 되지 않는 경우 학습률을 조정하여 모델이 조금 더 성능을 높일 수 있도록 도와주는 함수인 것이다.

 + monitor: ReduceOnPlatue의 기준이 되는 값  ex) val_loss(검증데이터셋의 loss)
 + factor: learning rate를 얼마나 감소시킬 것인지를 지정   ex) 현재 학습률이 0.1이고 factor가 0.5이면 새로운 학습률은 0.1 * 0.5 = 0.05
 + patience: 성능이 개선되지 않는 epoch를 몇 번 허용할 것인지를 지정
 
 
 <br>
 
 추가적으로, 콜백은 여러개를 선언을 해 놓고 `model.fit`을 할 때 리스트로 묶어주면 된다.

In [21]:
# define callbacks
earlystopping = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor = 'val_loss', mode = 'min')
reduce_lr_loss = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 10, verbose = 1, mode = 'min')

In [20]:
model = getModel()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 73, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 128)       1

In [22]:
model.fit(X_new_train, y_new_train, batch_size = 32, epochs = 50, verbose = 1, 
          callbacks = [earlystopping, mcp_save, reduce_lr_loss], validation_split = 0.3)

Epoch 1/50
106/106 [==============================] - 54s 495ms/step - loss: 0.6521 - accuracy: 0.5806 - val_loss: 0.4104 - val_accuracy: 0.8193
Epoch 2/50
106/106 [==============================] - 54s 509ms/step - loss: 0.4379 - accuracy: 0.7932 - val_loss: 0.3521 - val_accuracy: 0.8428
Epoch 3/50
106/106 [==============================] - 64s 606ms/step - loss: 0.3340 - accuracy: 0.8443 - val_loss: 0.3542 - val_accuracy: 0.8435
Epoch 4/50
106/106 [==============================] - 53s 496ms/step - loss: 0.3248 - accuracy: 0.8638 - val_loss: 0.3377 - val_accuracy: 0.8442
Epoch 5/50
106/106 [==============================] - 57s 534ms/step - loss: 0.2821 - accuracy: 0.8683 - val_loss: 0.3060 - val_accuracy: 0.8857
Epoch 6/50
106/106 [==============================] - 55s 519ms/step - loss: 0.2572 - accuracy: 0.8808 - val_loss: 0.2624 - val_accuracy: 0.8913
Epoch 7/50
106/106 [==============================] - 57s 536ms/step - loss: 0.2281 - accuracy: 0.9079 - val_loss: 0.2505 - val_ac

### 최종 모델 파라미터 가져오기
앞서 CNN Modeling + Callbacks 과정을 통해 가장 validation loss를 최소화하는 모델 파라미터를 ModelCheckpoint를 이용해 저장을 했다. 이제 마지막으로 저장한 파라미터를 불러와서 그대로 evaluation을 진행하면 된다.

In [24]:
# model evaluation
model.load_weights(filepath = '.mdl_wts.hdf5')

score = model.evaluate(X_new_train, y_new_train, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])

151/151 [==============================] - 15s 101ms/step - loss: 0.1429 - accuracy: 0.9453
Train score: 0.14286918938159943
Train accuracy: 0.9453449845314026
